# Benchmarking Agentic Qwen3 on HotpotQA

This notebook implements a **Reflexive Metacognitive Agent** powered by `qwen3:30b` (via Ollama) to solve multi-hop reasoning tasks from the HotpotQA dataset.

## Architecture Overview

The agent employs a self-correcting loop structure:

1. **Metacognitive Router** - Classifies query complexity (SIMPLE/COMPLEX)
2. **Planner** - Breaks down complex queries into search steps
3. **Executor** - Retrieves information from the HotpotQA context
4. **Reflector** - Validates retrieved information and decides next action
5. **Final Answer Synthesis** - Generates the final response

## Components

- **LangGraph** - Agent orchestration
- **Ollama** - Local LLM inference (qwen3:30b)
- **MLflow** - Tracing and experiment tracking
- **Google Gemini** - LLM-as-a-Judge evaluation


## 1. Setup & Dependencies

1. **Ollama**: Install from [ollama.ai](https://ollama.ai) and pull the model:
   ```bash
   ollama pull qwen3:30b
   ```

2. **MLflow Server** (for tracing): Start before running the benchmark:
   ```bash
   mlflow ui --port 5000
   ```

3. **Gemini API Key**: Add to `.env`


In [1]:
import os
import sys
from pathlib import Path
from typing import TypedDict, Literal, Annotated, List, Optional
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Load environment variables from .env file
# First try the project root, then current directory
project_root = Path().absolute().parent.parent
env_path = project_root / ".env"
if env_path.exists():
    load_dotenv(env_path)
    print(f"✓ Loaded .env from: {env_path}")
else:
    load_dotenv()
    print("✓ Loaded .env from current directory")

# Configuration
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "qwen3:30b")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_MODEL = "gemini-2.5-pro"

# Agent configuration
MAX_REFLECTIONS = 3  # Prevent infinite loops
SAMPLE_SIZE = 10  # Number of questions to evaluate
RANDOM_SEED = 42  # Seed for reproducible random sampling

print(f"\nConfiguration:")
print(f"  Ollama URL: {OLLAMA_BASE_URL}")
print(f"  Ollama Model: {OLLAMA_MODEL}")
print(f"  Gemini API Key: {'✓ Set' if GEMINI_API_KEY else '✗ Not set'}")
print(f"  Max Reflections: {MAX_REFLECTIONS}")
print(f"  Sample Size: {SAMPLE_SIZE}")
print(f"  Random Seed: {RANDOM_SEED}")


✓ Loaded .env from: /home/vincent/Code/zgx-onboard/.env

Configuration:
  Ollama URL: http://localhost:11434
  Ollama Model: qwen3:30b
  Gemini API Key: ✓ Set
  Max Reflections: 3
  Sample Size: 10
  Random Seed: 42


## 2. Connectivity Tests

Before proceeding, we verify connectivity to both the Ollama and Gemini models.


### 2.1 Ollama Connectivity Test


In [2]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

def test_ollama_connectivity(base_url: str, model: str) -> bool:
    """
    Test connectivity to Ollama server using LangChain's ChatOllama.
    This uses the same connection method as the agent.
    
    Args:
        base_url: Ollama server URL
        model: Model name to test
    
    Returns:
        bool: True if connection successful and model responds
    """
    try:
        print(f"Testing Ollama connection via LangChain ChatOllama...")
        print(f"  Base URL: {base_url}")
        print(f"  Model: {model}")
        
        # Initialize ChatOllama - same as the agent uses
        test_llm = ChatOllama(
            model=model,
            base_url=base_url,
            temperature=0.1,
        )
        
        # Run a simple inference test
        print(f"\n  Running inference test...")
        test_prompt = "What is the capital of France? Answer in one word."
        
        response = test_llm.invoke([HumanMessage(content=test_prompt)])
        answer = response.content.strip()[:100]
        
        print(f"  Prompt: '{test_prompt}'")
        print(f"  Response: '{answer}'")
        print(f"✓ Ollama (ChatOllama) connectivity test passed")
        
        return True
        
    except Exception as e:
        print(f"✗ Ollama connection failed: {e}")
        print(f"  Make sure Ollama is running: ollama serve")
        print(f"  And the model is pulled: ollama pull {model}")
        return False

# Run Ollama connectivity test
ollama_connected = test_ollama_connectivity(OLLAMA_BASE_URL, OLLAMA_MODEL)


Testing Ollama connection via LangChain ChatOllama...
  Base URL: http://localhost:11434
  Model: qwen3:30b

  Running inference test...
  Prompt: 'What is the capital of France? Answer in one word.'
  Response: 'Paris'
✓ Ollama (ChatOllama) connectivity test passed


### 2.2 Gemini Connectivity Test


In [3]:
import google.generativeai as genai

def test_gemini_connectivity(api_key: str, model: str = "gemini-2.5-pro") -> bool:
    """
    Test connectivity to Google Gemini API.
    
    Args:
        api_key: Google API key for Gemini
        model: Gemini model to use
    
    Returns:
        bool: True if connection successful
    """
    if not api_key:
        print("✗ Gemini API key not set")
        print("  Set GEMINI_API_KEY in your .env file")
        print("  Get your key at: https://aistudio.google.com/apikey")
        return False
    
    try:
        # Configure the API
        genai.configure(api_key=api_key)
        
        # List available models
        available_models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]
        print(f"✓ Gemini API connected")
        print(f"  Available models (sample): {', '.join(available_models[:5])}...")
        
        # Try to initialize the model
        # Handle model name variations
        model_name = model
        if not model_name.startswith("models/"):
            model_name = f"models/{model_name}"
        
        # Check if exact model exists, otherwise find a compatible one
        if model_name not in available_models:
            # Try to find a matching model
            matching = [m for m in available_models if model.split('-')[0] in m]
            if matching:
                model_name = matching[0]
                print(f"  Using model: {model_name}")
            else:
                # Fallback to gemini-pro
                model_name = "models/gemini-2.0-flash"
                print(f"  Falling back to: {model_name}")
        
        gemini_model = genai.GenerativeModel(model_name)
        
        # Run a simple test
        print(f"\n  Running inference test...")
        test_prompt = "What is the capital of France? Answer in one word."
        response = gemini_model.generate_content(test_prompt)
        answer = response.text.strip()[:100]
        print(f"  Prompt: '{test_prompt}'")
        print(f"  Response: '{answer}'")
        print(f"✓ Gemini inference test passed")
        
        return True
        
    except Exception as e:
        print(f"✗ Gemini connection failed: {e}")
        return False

# Run Gemini connectivity test
gemini_connected = test_gemini_connectivity(GEMINI_API_KEY, GEMINI_MODEL)


✓ Gemini API connected
  Available models (sample): models/gemini-2.5-pro-preview-03-25, models/gemini-2.5-flash, models/gemini-2.5-pro-preview-05-06, models/gemini-2.5-pro-preview-06-05, models/gemini-2.5-pro...

  Running inference test...
  Prompt: 'What is the capital of France? Answer in one word.'
  Response: 'Paris'
✓ Gemini inference test passed


In [4]:
# Summary of connectivity tests
print("\n" + "="*60)
print("CONNECTIVITY TEST SUMMARY")
print("="*60)
print(f"  Ollama ({OLLAMA_MODEL}): {'✓ Connected' if ollama_connected else '✗ Failed'}")
print(f"  Gemini ({GEMINI_MODEL}): {'✓ Connected' if gemini_connected else '✗ Failed'}")
print("="*60)

if not ollama_connected:
    print("\n⚠ Ollama is required for the agent. Please fix the connection before proceeding.")
if not gemini_connected:
    print("\n⚠ Gemini is required for evaluation. The agent will work but evaluation will fail.")



CONNECTIVITY TEST SUMMARY
  Ollama (qwen3:30b): ✓ Connected
  Gemini (gemini-2.5-pro): ✓ Connected


## 3. Load HotpotQA Dataset


In [5]:
from datasets import load_dataset
import random

# Load HotpotQA distractor dataset
print("Loading HotpotQA dataset...")
dataset = load_dataset("hotpot_qa", "distractor", split="validation")

print(f"✓ Dataset loaded")
print(f"  Total samples: {len(dataset)}")
print(f"  Features: {list(dataset.features.keys())}")

# Randomly sample a subset for testing (reproducible with seed)
random.seed(RANDOM_SEED)
sample_indices = random.sample(range(len(dataset)), min(SAMPLE_SIZE, len(dataset)))
sample_dataset = dataset.select(sample_indices)
print(f"  Sample size for evaluation: {len(sample_dataset)}")
print(f"  Random seed: {RANDOM_SEED}")
print(f"  Sampled indices: {sample_indices[:5]}{'...' if len(sample_indices) > 5 else ''}")


Loading HotpotQA dataset...
✓ Dataset loaded
  Total samples: 7405
  Features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context']
  Sample size for evaluation: 10
  Random seed: 42
  Sampled indices: [5238, 912, 204, 6074, 2253]...


In [6]:
# Examine a sample from the dataset
sample = dataset[0]
print("Sample Question:")
print(f"  ID: {sample['id']}")
print(f"  Question: {sample['question']}")
print(f"  Answer: {sample['answer']}")
print(f"  Type: {sample['type']}")
print(f"  Level: {sample['level']}")
print(f"\nContext (10 paragraphs - 2 gold, 8 distractors):")
for i, (title, sentences) in enumerate(zip(sample['context']['title'], sample['context']['sentences'])):
    preview = ' '.join(sentences)[:100] + "..."
    print(f"  [{i+1}] {title}: {preview}")


Sample Question:
  ID: 5a8b57f25542995d1e6f1371
  Question: Were Scott Derrickson and Ed Wood of the same nationality?
  Answer: yes
  Type: comparison
  Level: hard

Context (10 paragraphs - 2 gold, 8 distractors):
  [1] Ed Wood (film): Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton...
  [2] Scott Derrickson: Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.  He lives ...
  [3] Woodson, Arkansas: Woodson is a census-designated place (CDP) in Pulaski County, Arkansas, in the United States.  Its p...
  [4] Tyler Bates: Tyler Bates (born June 5, 1965) is an American musician, music producer, and composer for films, tel...
  [5] Ed Wood: Edward Davis Wood Jr. (October 10, 1924 – December 10, 1978) was an American filmmaker, actor, write...
  [6] Deliver Us from Evil (2014 film): Deliver Us from Evil is a 2014 American supernatural horror film directed by Scott Derrickson and pr...
  [7] Adam Coll

## 4. Document Search Environment

The agent is NOT given the context directly. Instead, it uses a `search_documents` tool that performs keyword search against the paragraphs hidden in the environment.


In [7]:
from dataclasses import dataclass, field
from difflib import SequenceMatcher
import re

@dataclass
class HotpotQAEnvironment:
    """
    Simulates a document retrieval environment for a HotpotQA question.
    The agent must use the search_documents tool to retrieve relevant context.
    """
    question: str
    answer: str
    paragraphs: List[dict] = field(default_factory=list)
    
    @classmethod
    def from_sample(cls, sample: dict) -> 'HotpotQAEnvironment':
        """Create environment from a HotpotQA sample."""
        paragraphs = []
        for title, sentences in zip(sample['context']['title'], sample['context']['sentences']):
            paragraphs.append({
                'title': title,
                'content': ' '.join(sentences)
            })
        return cls(
            question=sample['question'],
            answer=sample['answer'],
            paragraphs=paragraphs
        )
    
    def search_documents(self, query: str, top_k: int = 3) -> List[dict]:
        """
        Search for documents matching the query using keyword/fuzzy matching.
        
        Args:
            query: Search query
            top_k: Number of results to return
        
        Returns:
            List of matching paragraphs with relevance scores
        """
        results = []
        query_lower = query.lower()
        query_words = set(re.findall(r'\w+', query_lower))
        
        for para in self.paragraphs:
            title_lower = para['title'].lower()
            content_lower = para['content'].lower()
            full_text = f"{title_lower} {content_lower}"
            
            # Calculate relevance score
            # 1. Keyword overlap
            text_words = set(re.findall(r'\w+', full_text))
            keyword_overlap = len(query_words & text_words) / max(len(query_words), 1)
            
            # 2. Sequence matching for title
            title_similarity = SequenceMatcher(None, query_lower, title_lower).ratio()
            
            # 3. Check for exact phrase matches
            phrase_bonus = 0.3 if query_lower in full_text else 0
            
            # Combined score
            score = (keyword_overlap * 0.5) + (title_similarity * 0.3) + phrase_bonus
            
            if score > 0.1:  # Threshold
                results.append({
                    'title': para['title'],
                    'content': para['content'],
                    'relevance_score': round(score, 3)
                })
        
        # Sort by relevance and return top_k
        results.sort(key=lambda x: x['relevance_score'], reverse=True)
        return results[:top_k]

# Test the environment
test_env = HotpotQAEnvironment.from_sample(dataset[0])
print(f"Test Environment:")
print(f"  Question: {test_env.question}")
print(f"  Ground Truth: {test_env.answer}")
print(f"  Paragraphs: {len(test_env.paragraphs)}")

# Test search
test_query = test_env.question.split()[:5]  # Use first 5 words
test_query = ' '.join(test_query)
print(f"\nTest Search Query: '{test_query}'")
results = test_env.search_documents(test_query)
for r in results:
    print(f"  [{r['relevance_score']:.3f}] {r['title']}: {r['content'][:80]}...")


Test Environment:
  Question: Were Scott Derrickson and Ed Wood of the same nationality?
  Ground Truth: yes
  Paragraphs: 10

Test Search Query: 'Were Scott Derrickson and Ed'
  [0.518] Scott Derrickson: Scott Derrickson (born July 16, 1966) is an American director, screenwriter and ...
  [0.377] Adam Collis: Adam Collis is an American filmmaker and actor.  He attended the Duke University...
  [0.370] Deliver Us from Evil (2014 film): Deliver Us from Evil is a 2014 American supernatural horror film directed by Sco...


## 5. Agent State & Graph Definition

We define the agent's state schema and the LangGraph workflow implementing the Reflexive Metacognitive pattern.


In [8]:
from langgraph.graph import StateGraph, END, START
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
import operator
import json

# Define the agent state
class AgentState(TypedDict):
    """State schema for the Reflexive Metacognitive Agent."""
    messages: Annotated[List, operator.add]  # Chat history
    question: str  # Original question
    plan: List[str]  # Current list of search steps
    context: List[dict]  # Retrieved snippets
    reflection_count: int  # Counter to prevent infinite loops
    complexity: str  # SIMPLE or COMPLEX
    final_answer: Optional[str]  # The synthesized answer
    environment: Optional[HotpotQAEnvironment]  # The search environment

print("✓ AgentState defined")


✓ AgentState defined


In [9]:
# Initialize the Ollama LLM
llm = ChatOllama(
    model=OLLAMA_MODEL,
    base_url=OLLAMA_BASE_URL,
    temperature=0.1,
)

print(f"✓ LLM initialized: {OLLAMA_MODEL}")


✓ LLM initialized: qwen3:30b


In [10]:
import time as _time

# Node 1: Metacognitive Router
def router_node(state: AgentState) -> AgentState:
    """
    Analyzes the question complexity and routes accordingly.
    Output: SIMPLE or COMPLEX
    """
    print("  ├─ 🔀 [Router] Analyzing question complexity...")
    start = _time.time()
    
    question = state["question"]
    
    router_prompt = f"""Analyze this question and determine its complexity.

Question: {question}

Is this a SIMPLE lookup question (can be answered with a single fact) or a COMPLEX multi-step reasoning question (requires combining information from multiple sources)?

Output ONLY one word: SIMPLE or COMPLEX"""
    
    llm_start = _time.time()
    response = llm.invoke([HumanMessage(content=router_prompt)])
    llm_time = _time.time() - llm_start
    
    complexity = "COMPLEX" if "COMPLEX" in response.content.upper() else "SIMPLE"
    total_time = _time.time() - start
    
    print(f"  │    └─ Classified as: {complexity} (LLM: {llm_time:.2f}s, total: {total_time:.2f}s)")
    
    return {
        **state,
        "complexity": complexity,
        "messages": [AIMessage(content=f"[Router] Classified as: {complexity}")]
    }

print("✓ Router node defined")


✓ Router node defined


In [11]:
# Node 2: Planner
def planner_node(state: AgentState) -> AgentState:
    """
    Breaks down complex queries into search steps.
    Generates specific search queries for the retrieval tool.
    """
    reflection_count = state.get("reflection_count", 0)
    print(f"  ├─ 📋 [Planner] Generating search queries (iteration {reflection_count + 1})...")
    start = _time.time()
    
    question = state["question"]
    current_context = state.get("context", [])
    
    # Build context summary if we have prior retrievals
    context_summary = ""
    if current_context:
        context_summary = "\n\nInformation already retrieved:\n"
        for ctx in current_context:
            context_summary += f"- {ctx['title']}: {ctx['content'][:100]}...\n"
        print(f"  │    ├─ Prior context: {len(current_context)} documents")
    
    planner_prompt = f"""You are a research planner. Your task is to create search queries to find information needed to answer a question.

Question: {question}
{context_summary}

Generate 1-3 specific search queries that would help find the missing information. Each query should focus on a specific entity or fact mentioned in the question.

Output as a JSON array of strings, e.g.: ["query 1", "query 2"]

Search queries:"""
    
    llm_start = _time.time()
    response = llm.invoke([HumanMessage(content=planner_prompt)])
    llm_time = _time.time() - llm_start
    
    # Parse the response to extract queries
    try:
        # Try to extract JSON array
        content = response.content
        # Find JSON array in response
        start_idx = content.find('[')
        end_idx = content.rfind(']') + 1
        if start_idx != -1 and end_idx > start_idx:
            queries = json.loads(content[start_idx:end_idx])
        else:
            # Fallback: split by newlines
            queries = [q.strip('- ') for q in content.split('\n') if q.strip()]
    except:
        # Ultimate fallback: use the question itself
        queries = [question]
    
    total_time = _time.time() - start
    print(f"  │    └─ Generated {len(queries[:3])} queries (LLM: {llm_time:.2f}s, total: {total_time:.2f}s)")
    for i, q in enumerate(queries[:3]):
        print(f"  │       [{i+1}] {q[:60]}{'...' if len(q) > 60 else ''}")
    
    return {
        **state,
        "plan": queries[:3],  # Limit to 3 queries
        "messages": [AIMessage(content=f"[Planner] Generated {len(queries)} search queries: {queries}")]
    }

print("✓ Planner node defined")


✓ Planner node defined


In [12]:
# Node 3: Executor (Tool Node)
def executor_node(state: AgentState) -> AgentState:
    """
    Executes the search plan by querying the document environment.
    Simulates retrieval from the HotpotQA context.
    """
    print("  ├─ 🔍 [Executor] Executing search queries...")
    start = _time.time()
    
    plan = state.get("plan", [])
    environment = state.get("environment")
    current_context = state.get("context", [])
    
    if not environment:
        print("  │    └─ ❌ Error: No environment set")
        return {
            **state,
            "messages": [AIMessage(content="[Executor] Error: No environment set")]
        }
    
    # Execute each query in the plan
    new_context = list(current_context)
    retrieved_titles = {ctx['title'] for ctx in new_context}
    new_docs_count = 0
    
    for i, query in enumerate(plan):
        query_start = _time.time()
        results = environment.search_documents(query, top_k=2)
        query_time = _time.time() - query_start
        added = 0
        for result in results:
            # Avoid duplicates
            if result['title'] not in retrieved_titles:
                new_context.append(result)
                retrieved_titles.add(result['title'])
                added += 1
                new_docs_count += 1
        print(f"  │    ├─ Query {i+1}: found {len(results)} docs, added {added} new ({query_time*1000:.1f}ms)")
    
    total_time = _time.time() - start
    print(f"  │    └─ Total: {len(new_context)} docs in context (+{new_docs_count} new) ({total_time:.2f}s)")
    
    return {
        **state,
        "context": new_context,
        "messages": [AIMessage(content=f"[Executor] Retrieved {len(new_context)} documents")]
    }

print("✓ Executor node defined")


✓ Executor node defined


In [13]:
# Node 4: Reflector (Critic)
def reflector_node(state: AgentState) -> AgentState:
    """
    Validates retrieved information against the user query.
    Decides whether to continue searching or generate final answer.
    """
    reflection_count = state.get("reflection_count", 0)
    print(f"  ├─ 🤔 [Reflector] Evaluating information sufficiency (reflection #{reflection_count + 1}/{MAX_REFLECTIONS})...")
    start = _time.time()
    
    question = state["question"]
    context = state.get("context", [])
    
    # Build context text
    context_text = "\n\n".join([
        f"**{ctx['title']}**: {ctx['content']}"
        for ctx in context
    ])
    
    reflector_prompt = f"""You are a critical reviewer. Analyze whether the retrieved information is sufficient to answer the question.

Question: {question}

Retrieved Information:
{context_text}

Based ONLY on the retrieved information:
1. Can you confidently answer the question?
2. Is there any critical information missing?

Output your assessment in this format:
SUFFICIENT: YES or NO
MISSING: [what information is still needed, if any]
REASONING: [brief explanation]"""
    
    llm_start = _time.time()
    response = llm.invoke([HumanMessage(content=reflector_prompt)])
    llm_time = _time.time() - llm_start
    content = response.content.upper()
    
    # Determine if we have sufficient information
    is_sufficient = "SUFFICIENT: YES" in content or "SUFFICIENT:YES" in content
    
    # Increment reflection count
    new_reflection_count = reflection_count + 1
    
    # Force completion if max reflections reached
    forced = False
    if new_reflection_count >= MAX_REFLECTIONS and not is_sufficient:
        is_sufficient = True
        forced = True
    
    total_time = _time.time() - start
    status = "✅ Sufficient" if is_sufficient else "🔄 Need more info"
    if forced:
        status = "⚠️ Max reflections reached, forcing completion"
    print(f"  │    └─ {status} (LLM: {llm_time:.2f}s, total: {total_time:.2f}s)")
    
    return {
        **state,
        "reflection_count": new_reflection_count,
        "complexity": "DONE" if is_sufficient else "CONTINUE",
        "messages": [AIMessage(content=f"[Reflector] Reflection #{new_reflection_count}: {'Sufficient' if is_sufficient else 'Need more info'}")]
    }

print("✓ Reflector node defined")


✓ Reflector node defined


In [14]:
# Node 5: Direct Answer (for SIMPLE queries)
def direct_answer_node(state: AgentState) -> AgentState:
    """
    Generates a quick answer for simple queries with minimal retrieval.
    """
    print("  ├─ ⚡ [DirectAnswer] Generating quick answer for simple query...")
    start = _time.time()
    
    question = state["question"]
    environment = state.get("environment")
    
    # Quick retrieval
    if environment:
        search_start = _time.time()
        results = environment.search_documents(question, top_k=3)
        search_time = _time.time() - search_start
        context_text = "\n".join([f"- {r['title']}: {r['content'][:200]}" for r in results])
        print(f"  │    ├─ Quick search: {len(results)} docs ({search_time*1000:.1f}ms)")
    else:
        context_text = "No context available."
        results = []
        print("  │    ├─ No environment, proceeding without context")
    
    answer_prompt = f"""Answer the following question based on the provided context. Be concise and direct.

Question: {question}

Context:
{context_text}

Answer:"""
    
    llm_start = _time.time()
    response = llm.invoke([HumanMessage(content=answer_prompt)])
    llm_time = _time.time() - llm_start
    
    total_time = _time.time() - start
    print(f"  │    └─ Answer generated (LLM: {llm_time:.2f}s, total: {total_time:.2f}s)")
    
    return {
        **state,
        "final_answer": response.content.strip(),
        "context": results if environment else [],
        "messages": [AIMessage(content=f"[DirectAnswer] Generated answer")]
    }

print("✓ Direct Answer node defined")


✓ Direct Answer node defined


In [15]:
# Node 6: Final Answer Synthesis
def final_answer_node(state: AgentState) -> AgentState:
    """
    Synthesizes the final answer from all retrieved context.
    """
    print("  ├─ ✨ [FinalAnswer] Synthesizing final answer...")
    start = _time.time()
    
    question = state["question"]
    context = state.get("context", [])
    
    print(f"  │    ├─ Using {len(context)} context documents")
    
    # Build comprehensive context
    context_text = "\n\n".join([
        f"**{ctx['title']}**:\n{ctx['content']}"
        for ctx in context
    ])
    
    synthesis_prompt = f"""You are an expert at synthesizing information to answer complex questions.

Question: {question}

Retrieved Information:
{context_text}

Based on the information above, provide a clear and accurate answer to the question. 
Be specific and cite relevant facts from the sources.
If the information is insufficient, say so honestly.

Answer:"""
    
    llm_start = _time.time()
    response = llm.invoke([HumanMessage(content=synthesis_prompt)])
    llm_time = _time.time() - llm_start
    
    total_time = _time.time() - start
    print(f"  │    └─ Final answer synthesized (LLM: {llm_time:.2f}s, total: {total_time:.2f}s)")
    
    return {
        **state,
        "final_answer": response.content.strip(),
        "messages": [AIMessage(content=f"[FinalAnswer] Synthesized final response")]
    }

print("✓ Final Answer node defined")


✓ Final Answer node defined


In [16]:
# Define routing functions
def route_by_complexity(state: AgentState) -> Literal["planner", "direct_answer"]:
    """Route based on question complexity."""
    if state.get("complexity") == "SIMPLE":
        return "direct_answer"
    return "planner"

def route_after_reflection(state: AgentState) -> Literal["planner", "final_answer"]:
    """Route based on reflection result."""
    if state.get("complexity") == "DONE":
        return "final_answer"
    return "planner"

print("✓ Routing functions defined")


✓ Routing functions defined


In [17]:
# Build the LangGraph workflow
def build_agent_graph() -> StateGraph:
    """
    Constructs the Reflexive Metacognitive Agent graph.
    
    Flow:
    START -> Router -> [SIMPLE: DirectAnswer | COMPLEX: Planner -> Executor -> Reflector]
                                                                    |-> loop back to Planner
                                                                    |-> FinalAnswer -> END
    """
    # Create the graph
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("router", router_node)
    workflow.add_node("planner", planner_node)
    workflow.add_node("executor", executor_node)
    workflow.add_node("reflector", reflector_node)
    workflow.add_node("direct_answer", direct_answer_node)
    workflow.add_node("final_answer", final_answer_node)
    
    # Add edges
    workflow.add_edge(START, "router")
    workflow.add_conditional_edges(
        "router",
        route_by_complexity,
        {
            "planner": "planner",
            "direct_answer": "direct_answer"
        }
    )
    workflow.add_edge("planner", "executor")
    workflow.add_edge("executor", "reflector")
    workflow.add_conditional_edges(
        "reflector",
        route_after_reflection,
        {
            "planner": "planner",
            "final_answer": "final_answer"
        }
    )
    workflow.add_edge("direct_answer", END)
    workflow.add_edge("final_answer", END)
    
    return workflow.compile()

# Build the agent
agent = build_agent_graph()
print("✓ Agent graph compiled")


✓ Agent graph compiled


## 6. Test the Agent


In [18]:
def run_agent(question: str, environment: HotpotQAEnvironment, verbose: bool = True) -> dict:
    """
    Run the agent on a single question.
    
    Args:
        question: The question to answer
        environment: The HotpotQA environment with context
        verbose: Whether to print detailed progress (default True)
    
    Returns:
        Final state with answer
    """
    if verbose:
        print("  ┌─ 🚀 Agent workflow started")
        print(f"  │  Question: {question[:70]}{'...' if len(question) > 70 else ''}")
    
    start_time = _time.time()
    
    initial_state = {
        "messages": [],
        "question": question,
        "plan": [],
        "context": [],
        "reflection_count": 0,
        "complexity": "",
        "final_answer": None,
        "environment": environment
    }
    
    result = agent.invoke(initial_state)
    
    total_time = _time.time() - start_time
    
    if verbose:
        print(f"  └─ ✅ Agent workflow completed in {total_time:.2f}s")
    
    return result

# Test with first sample
test_sample = dataset[0]
test_env = HotpotQAEnvironment.from_sample(test_sample)

print(f"Question: {test_sample['question']}")
print(f"Ground Truth: {test_sample['answer']}")
print(f"\n{'='*60}")
print("RUNNING AGENT")
print(f"{'='*60}")

result = run_agent(test_sample['question'], test_env)

print(f"\n{'='*60}")
print(f"AGENT TRACE (message log)")
print(f"{'='*60}")
for msg in result['messages']:
    print(f"  {msg.content}")

print(f"\n{'='*60}")
print(f"RESULTS")
print(f"{'='*60}")
print(f"Complexity: {result['complexity']}")
print(f"Reflections: {result['reflection_count']}")
print(f"Context Retrieved: {len(result['context'])} documents")
print(f"\nAgent Answer:")
print(result['final_answer'])
print(f"\nGround Truth: {test_sample['answer']}")


Question: Were Scott Derrickson and Ed Wood of the same nationality?
Ground Truth: yes

RUNNING AGENT
  ┌─ 🚀 Agent workflow started
  │  Question: Were Scott Derrickson and Ed Wood of the same nationality?
  ├─ 🔀 [Router] Analyzing question complexity...
  │    └─ Classified as: COMPLEX (LLM: 20.44s, total: 20.44s)
  ├─ 📋 [Planner] Generating search queries (iteration 1)...
  │    └─ Generated 2 queries (LLM: 6.86s, total: 6.86s)
  │       [1] Scott Derrickson nationality
  │       [2] Ed Wood nationality
  ├─ 🔍 [Executor] Executing search queries...
  │    ├─ Query 1: found 2 docs, added 2 new (0.6ms)
  │    ├─ Query 2: found 2 docs, added 2 new (0.4ms)
  │    └─ Total: 4 docs in context (+4 new) (0.00s)
  ├─ 🤔 [Reflector] Evaluating information sufficiency (reflection #1/3)...
  │    └─ ✅ Sufficient (LLM: 12.86s, total: 12.86s)
  ├─ ✨ [FinalAnswer] Synthesizing final answer...
  │    ├─ Using 4 context documents
  │    └─ Final answer synthesized (LLM: 6.96s, total: 6.96s)
  └─ ✅ Age

## 7. MLflow Tracing & Evaluation Setup

### Prerequisites: Start the MLflow Tracking Server

Before running the benchmark, start the MLflow UI server in a separate terminal to view traces and experiments:

```bash
# Option 1: Start MLflow UI with default settings (uses ./mlruns directory)
mlflow ui

# Option 2: Specify a custom port
mlflow ui --port 5000

# Option 3: Start with a specific backend store
mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns
```

The MLflow UI will be available at **http://localhost:5000** (or your specified port).

### What Gets Traced?

With `mlflow.langchain.autolog()` enabled, the following will be automatically captured:
- **LLM calls**: Every invocation of `ChatOllama` (router, planner, reflector, etc.)
- **Input/Output**: Prompts sent and responses received
- **Latency**: Time taken for each LLM call
- **Token counts**: Input and output token estimates
- **Chain execution**: The full LangGraph workflow execution


In [19]:
import mlflow

# Configure MLflow
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI", "file:./mlruns")
EXPERIMENT_NAME = "hotpotqa-qwen3-benchmark"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

# Enable LangChain autologging for tracing (requires 'langchain' package)
try:
    mlflow.langchain.autolog()
    print("✓ LangChain autologging enabled")
except Exception as e:
    print(f"⚠ LangChain autologging not available: {e}")
    print("  MLflow will still track runs and metrics, but without automatic LLM tracing.")

print(f"\n✓ MLflow configured")
print(f"  Tracking URI: {MLFLOW_TRACKING_URI}")
print(f"  Experiment: {EXPERIMENT_NAME}")


2025/11/29 21:30:47 INFO mlflow.tracking.fluent: Experiment with name 'hotpotqa-qwen3-benchmark' does not exist. Creating a new experiment.


✓ LangChain autologging enabled

✓ MLflow configured
  Tracking URI: file:./mlruns
  Experiment: hotpotqa-qwen3-benchmark


### Accessing Traces in the MLflow UI

After running experiments, open **http://localhost:5000** in your browser to access the MLflow UI:

1. **Select Experiment**: Click on `hotpotqa-qwen3-benchmark` in the left sidebar
2. **View Runs**: Each benchmark execution creates a new run with logged metrics
3. **Explore Traces**: Click on a run, then navigate to the **"Traces"** tab to see:
   - Full execution timeline of the agent
   - Individual LLM calls with prompts and responses
   - Latency breakdown per node (Router → Planner → Executor → Reflector)
4. **Compare Runs**: Select multiple runs to compare metrics across different configurations

**Key Metrics to Monitor:**
| Metric | Description |
|--------|-------------|
| `avg_score` | Average evaluation score (1-5) from Gemini judge |
| `avg_latency` | Average time per question |
| `success_rate` | Percentage of successfully answered questions |
| `avg_reflections` | Average number of retrieval-reflection cycles |


**Tip**: If using `file:./mlruns` (default), run `mlflow ui` from the notebook's directory to see the experiments.


In [20]:
# Configure environment for MLflow's make_judge API
# The make_judge API requires the GEMINI_API_KEY to be set in the environment
# Reference: https://mlflow.org/docs/latest/genai/eval-monitor/scorers/llm-judge/make-judge/

import os

def verify_judge_configuration():
    """
    Verifies that the environment is configured for MLflow's template-based LLM Scorer.
    MLflow's make_judge uses the model URI format (e.g., 'gemini/gemini-2.5-pro')
    and reads credentials from environment variables.
    """
    if not GEMINI_API_KEY:
        print("⚠ GEMINI_API_KEY not set. Evaluation will use fallback scoring.")
        return False
    
    # Ensure GEMINI_API_KEY is available to MLflow's judge
    os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
    
    print(f"✓ MLflow LLM Judge environment configured")
    print(f"  Using model: gemini/{GEMINI_MODEL}")
    print(f"  API: MLflow make_judge (template-based LLM Scorer)")
    return True

gemini_judge_ready = verify_judge_configuration()


✓ MLflow LLM Judge environment configured
  Using model: gemini/gemini-2.5-pro
  API: MLflow make_judge (template-based LLM Scorer)


In [21]:
from mlflow.genai.judges import make_judge
from typing import Literal

# Create template-based LLM Scorer using MLflow's make_judge API
# Reference: https://mlflow.org/docs/latest/genai/eval-monitor/scorers/llm-judge/make-judge/

# Define the answer quality judge using MLflow's template-based approach
answer_quality_judge = make_judge(
    name="answer_quality",
    instructions=(
        "You are an expert judge evaluating question-answering quality.\n\n"
        "Question: {{ inputs }}\n"
        "Ground Truth Answer: {{ expectations }}\n"
        "Agent's Answer: {{ outputs }}\n\n"
        "Rate the agent's answer based on how well it matches the ground truth.\n"
        "Consider correctness, completeness, and relevance.\n"
        "- 'perfect': Matches or exceeds ground truth\n"
        "- 'correct': Correct with minor issues\n"
        "- 'mostly_correct': Mostly correct but missing key details\n"
        "- 'partial': Partially correct but with significant errors\n"
        "- 'wrong': Completely wrong or unrelated\n"
    ),
    feedback_value_type=Literal["perfect", "correct", "mostly_correct", "partial", "wrong"],
    model=f"gemini:/{GEMINI_MODEL}",
)

# Score mapping for metrics
SCORE_MAP = {
    "perfect": 5,
    "correct": 4,
    "mostly_correct": 3,
    "partial": 2,
    "wrong": 1,
}

def evaluate_answer(agent_answer: str, ground_truth: str, question: str, judge_model=None) -> dict:
    """
    Evaluate agent answer against ground truth using MLflow's template-based LLM judge.
    
    Args:
        agent_answer: The agent's generated answer
        ground_truth: The correct answer
        question: The original question
        judge_model: Legacy parameter (kept for compatibility, now uses make_judge)
    
    Returns:
        dict with score (1-5) and explanation
    """
    if not GEMINI_API_KEY:
        # Fallback: simple string matching
        gt_lower = ground_truth.lower().strip()
        ans_lower = agent_answer.lower().strip()
        if gt_lower in ans_lower or ans_lower in gt_lower:
            return {"score": 4, "explanation": "Fallback: Answer contains ground truth"}
        return {"score": 2, "explanation": "Fallback: Unable to verify without judge"}
    
    try:
        # Use MLflow's template-based judge
        feedback = answer_quality_judge(
            inputs={"question": question},
            outputs={"answer": agent_answer},
            expectations={"ground_truth": ground_truth},
        )
        
        # Convert categorical feedback to numeric score
        score = SCORE_MAP.get(feedback.value, 3)
        explanation = feedback.rationale or "No rationale provided"
        
        return {"score": score, "explanation": explanation}
    except Exception as e:
        return {"score": 0, "explanation": f"Evaluation error: {str(e)}"}

print("✓ MLflow template-based LLM Scorer (make_judge) configured")
print(f"  Model: gemini/{GEMINI_MODEL}")
print(f"  Feedback categories: perfect, correct, mostly_correct, partial, wrong")


✓ MLflow template-based LLM Scorer (make_judge) configured
  Model: gemini/gemini-2.5-pro
  Feedback categories: perfect, correct, mostly_correct, partial, wrong


## 8. Run Benchmark


In [22]:
from datetime import datetime
import time
import pandas as pd
from mlflow.entities import Feedback, Expectation, AssessmentSource

def run_benchmark(dataset, sample_size: int = 10, verbose: bool = True, random_seed: int = None) -> pd.DataFrame:
    """
    Run the full benchmark on HotpotQA samples.
    
    Uses MLflow's template-based LLM Scorer (make_judge) for evaluation.
    Logs per-trace assessments (Feedback) for each request so metrics appear at trace level in MLflow UI.
    
    Reference: https://mlflow.org/docs/latest/genai/eval-monitor/scorers/llm-judge/make-judge/
    
    Args:
        dataset: HotpotQA dataset
        sample_size: Number of samples to evaluate
        verbose: Whether to print detailed agent progress (default True)
        random_seed: Seed for random sampling (default uses RANDOM_SEED from config)
    
    Returns:
        DataFrame with results
    """
    results = []
    
    # Use random sampling to select cases from the dataset
    seed = random_seed if random_seed is not None else RANDOM_SEED
    random.seed(seed)
    total_available = len(dataset)
    actual_sample_size = min(sample_size, total_available)
    sampled_indices = random.sample(range(total_available), actual_sample_size)
    
    with mlflow.start_run(run_name=f"benchmark_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
        # Log parameters
        mlflow.log_params({
            "model": OLLAMA_MODEL,
            "sample_size": actual_sample_size,
            "max_reflections": MAX_REFLECTIONS,
            "judge_model": f"gemini/{GEMINI_MODEL}",
            "judge_type": "mlflow_make_judge",
            "random_seed": seed,
            "sampling_method": "random"
        })
        
        print(f"\n{'='*70}")
        print(f"🏃 BENCHMARK STARTED: {actual_sample_size} samples (randomly selected)")
        print(f"{'='*70}")
        print(f"Model: {OLLAMA_MODEL} | Max Reflections: {MAX_REFLECTIONS}")
        print(f"Judge: MLflow make_judge (gemini/{GEMINI_MODEL})")
        print(f"Random Seed: {seed} | Dataset Size: {total_available}")
        print(f"{'='*70}")
        
        benchmark_start = time.time()
        
        for i, dataset_idx in enumerate(sampled_indices):
            sample = dataset[dataset_idx]
            env = HotpotQAEnvironment.from_sample(sample)
            
            print(f"\n{'─'*70}")
            print(f"📝 Question [{i+1}/{actual_sample_size}] (idx={dataset_idx}): {sample['question']}")
            print(f"   Ground Truth: {sample['answer']}")
            print(f"{'─'*70}")
            
            # Run agent
            start_time = time.time()
            try:
                result = run_agent(sample['question'], env, verbose=verbose)
                latency = time.time() - start_time
                agent_answer = result.get('final_answer', '')
                success = True
            except Exception as e:
                latency = time.time() - start_time
                agent_answer = f"Error: {str(e)}"
                result = {"complexity": "ERROR", "reflection_count": 0, "context": []}
                success = False
                print(f"  ❌ Agent error: {str(e)}")
            
            # Get the trace_id for this request (created by langchain autolog)
            trace_id = mlflow.get_last_active_trace_id()
            
            # Evaluate using MLflow's template-based LLM Scorer
            print(f"\n  📊 Evaluating with MLflow make_judge (gemini/{GEMINI_MODEL})...")
            eval_start = time.time()
            evaluation = evaluate_answer(
                agent_answer, 
                sample['answer'], 
                sample['question']
            )
            eval_time = time.time() - eval_start
            
            # Log per-trace assessments (Feedback) so they appear in MLflow UI at trace level
            if trace_id:
                try:
                    # Log score as feedback
                    score_feedback = Feedback(
                        name="score",
                        value=evaluation['score'],
                        rationale=evaluation['explanation'][:500] if evaluation['explanation'] else None,
                        source=AssessmentSource(source_type="LLM_JUDGE", source_id=f"gemini/{GEMINI_MODEL}"),
                        metadata={"max_score": "5", "question_id": sample['id']},
                    )
                    mlflow.log_assessment(trace_id=trace_id, assessment=score_feedback)
                    
                    # Log latency as feedback
                    latency_feedback = Feedback(
                        name="latency_seconds",
                        value=round(latency, 3),
                        source=AssessmentSource(source_type="CODE", source_id="benchmark"),
                    )
                    mlflow.log_assessment(trace_id=trace_id, assessment=latency_feedback)
                    
                    # Log success as feedback
                    success_feedback = Feedback(
                        name="success",
                        value=success,
                        source=AssessmentSource(source_type="CODE", source_id="benchmark"),
                    )
                    mlflow.log_assessment(trace_id=trace_id, assessment=success_feedback)
                    
                    # Log reflections count as feedback
                    reflections_feedback = Feedback(
                        name="reflections",
                        value=result.get('reflection_count', 0),
                        source=AssessmentSource(source_type="CODE", source_id="benchmark"),
                    )
                    mlflow.log_assessment(trace_id=trace_id, assessment=reflections_feedback)
                    
                    # Log ground truth as expectation
                    expectation = Expectation(
                        name="expected_answer",
                        value=sample['answer'],
                        source=AssessmentSource(source_type="HUMAN", source_id="hotpotqa_dataset"),
                    )
                    mlflow.log_assessment(trace_id=trace_id, assessment=expectation)
                    
                    print(f"  📝 Logged assessments to trace: {trace_id[:20]}...")
                except Exception as e:
                    print(f"  ⚠️ Failed to log trace assessments: {e}")
            else:
                print(f"  ⚠️ No trace_id captured for this request")
            
            # Store result
            results.append({
                "dataset_index": dataset_idx,
                "question_id": sample['id'],
                "question": sample['question'],
                "ground_truth": sample['answer'],
                "agent_answer": agent_answer,
                "complexity": result.get('complexity', 'UNKNOWN'),
                "reflections": result.get('reflection_count', 0),
                "context_count": len(result.get('context', [])),
                "latency_seconds": latency,
                "score": evaluation['score'],
                "evaluation": evaluation['explanation'][:200],
                "success": success,
                "trace_id": trace_id,
            })
            
            # Summary for this question
            score_emoji = "🌟" if evaluation['score'] >= 4 else "⭐" if evaluation['score'] >= 3 else "💫"
            print(f"\n  {score_emoji} Score: {evaluation['score']}/5")
            print(f"  ⏱️  Agent latency: {latency:.2f}s | Evaluation: {eval_time:.2f}s")
            print(f"  📄 Context docs: {len(result.get('context', []))} | Reflections: {result.get('reflection_count', 0)}")
            
            # Running stats
            df_temp = pd.DataFrame(results)
            elapsed = time.time() - benchmark_start
            avg_per_q = elapsed / (i + 1)
            remaining = avg_per_q * (actual_sample_size - i - 1)
            print(f"  📈 Running avg score: {df_temp['score'].mean():.2f}/5 | Est. remaining: {remaining/60:.1f}min")
        
        # Create DataFrame
        df = pd.DataFrame(results)
        
        total_time = time.time() - benchmark_start
        
        # Log run-level aggregate metrics
        mlflow.log_metrics({
            "avg_score": df['score'].mean(),
            "avg_latency": df['latency_seconds'].mean(),
            "success_rate": df['success'].mean(),
            "avg_reflections": df['reflections'].mean(),
            "total_time_s": total_time
        })
        
        # Save results
        output_file = f"benchmark_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(output_file, index=False)
        mlflow.log_artifact(output_file)
        
        print(f"\n{'='*70}")
        print("🏆 BENCHMARK COMPLETE")
        print(f"{'='*70}")
        print(f"  📊 Samples Evaluated: {len(df)}")
        print(f"  ⭐ Average Score: {df['score'].mean():.2f}/5")
        print(f"  ⏱️  Average Latency: {df['latency_seconds'].mean():.2f}s per question")
        print(f"  ✅ Success Rate: {df['success'].mean()*100:.1f}%")
        print(f"  🔄 Avg Reflections: {df['reflections'].mean():.1f}")
        print(f"  ⏳ Total Time: {total_time/60:.1f} minutes")
        print(f"  💾 Results saved to: {output_file}")
        print(f"  📋 Trace assessments logged for each request (view in MLflow UI → Traces tab)")
        print(f"{'='*70}")
        
        return df

print("✓ Benchmark function defined (with per-trace assessments)")


✓ Benchmark function defined (with per-trace assessments)


In [23]:
# Run the benchmark
# Uses MLflow's template-based LLM Scorer (make_judge) for evaluation
# Cases are randomly selected from the dataset (controlled by RANDOM_SEED for reproducibility)
benchmark_results = run_benchmark(dataset, sample_size=SAMPLE_SIZE)


🏃 BENCHMARK STARTED: 10 samples (randomly selected)
Model: qwen3:30b | Max Reflections: 3
Judge: MLflow make_judge (gemini/gemini-2.5-pro)
Random Seed: 42 | Dataset Size: 7405

──────────────────────────────────────────────────────────────────────
📝 Question [1/10] (idx=5238): In what year was the university where Sergei Aleksandrovich Tokarev was a professor founded?
   Ground Truth: 1755
──────────────────────────────────────────────────────────────────────
  ┌─ 🚀 Agent workflow started
  │  Question: In what year was the university where Sergei Aleksandrovich Tokarev wa...
  ├─ 🔀 [Router] Analyzing question complexity...
  │    └─ Classified as: COMPLEX (LLM: 14.46s, total: 14.46s)
  ├─ 📋 [Planner] Generating search queries (iteration 1)...
  │    └─ Generated 0 queries (LLM: 702.03s, total: 702.03s)
  ├─ 🔍 [Executor] Executing search queries...
  │    └─ Total: 0 docs in context (+0 new) (0.00s)
  ├─ 🤔 [Reflector] Evaluating information sufficiency (reflection #1/3)...
  │    └─ 🔄

In [24]:
# Display results
if 'benchmark_results' in dir() and benchmark_results is not None:
    display(benchmark_results[['question', 'ground_truth', 'agent_answer', 'score', 'latency_seconds']])

,question,ground_truth,agent_answer,score,latency_seconds
0,In what year was the university where Sergei A...,1755,Based solely on the provided retrieved informa...,5,796.530573
1,Black Book starred the actress and writer of w...,Dutch,Dutch,5,48.082319
2,Which actor does American Beauty and American ...,Kevin Spacey,Kevin Spacey,5,38.347693
3,Ken Pruitt was a Republican member of an uppe...,40 members,Based solely on the provided retrieved informa...,5,48.739338
4,"Between Greyia and Calibanus, which genus cont...",Greyia,Based solely on the provided retrieved informa...,5,61.506555
5,Did John Updike and Tom Clancy both publish mo...,yes,Based solely on the retrieved information prov...,1,177.159528
6,Who was hung for assisting the attempted surre...,John André,John André,5,22.517682
7,which Mexican and American film actress is Eth...,Salma Hayek Pinault,Salma Hayek,4,15.664922
8,Which major international airport in south-eas...,Gatwick Airport,Based solely on the provided retrieved informa...,5,75.777518
9,Isabella Kelly was born at a ruined castle cha...,The Changing Scottish Landscape,Based solely on the retrieved information prov...,5,101.473732
